In [1]:
import itertools
import json
from collections import Counter

import numpy as np

In [4]:
def parse_input(path):
    with open(path) as file_in:
        data = file_in.read().splitlines()

    data = [tuple(map(int, row.split(","))) for row in data]
    data = [(y, x) for x, y in data]
    data = sorted(data, key=lambda x: x[0])

    min_row = min(data, key=lambda t: t[0])[0]
    max_row = max(data, key=lambda t: t[0])[0]

    return data, min_row, max_row


def compute_ranges_from_red_tiles(red_tiles, by):

    if by == "col":
        red_tiles = [(y, x) for x, y in red_tiles]
    red_tiles = sorted(red_tiles, key=lambda x: x[0])

    row_ranges = {}
    for i in range(0, len(red_tiles)-1, 2):
        bottom = min(red_tiles[i][1], red_tiles[i+1][1])
        top = max(red_tiles[i][1], red_tiles[i+1][1])
        row_ranges[red_tiles[i][0]] = range(bottom, top+1)
    return row_ranges


def get_complete_range_row(row, red_tiles_row_ranges, red_tiles_col_ranges):

    if row in red_tiles_row_ranges:
        min_row = red_tiles_row_ranges[row].start
        max_row = red_tiles_row_ranges[row].stop-1
    else:
        min_row = float('inf')
        max_row = 0

    for col in red_tiles_col_ranges:
        if row in red_tiles_col_ranges[col]:
            min_row = min(min_row, col)
            max_row = max(max_row, col)

    if min_row == float('inf') and max_row == 0:
        return None
    else:
        return range(min_row, max_row+1)


def get_all_complete_range_rows(red_tiles):
    row_ranges_between_red_tiles = compute_ranges_from_red_tiles(red_tiles, by="row")
    col_ranges_between_red_tiles = compute_ranges_from_red_tiles(red_tiles, by="col")
    
    min_row = min(red_tiles, key=lambda t: t[0])[0]
    max_row = max(red_tiles, key=lambda t: t[0])[0]
    dict_complete_range_rows = {}
    for row in range(min_row, max_row+1):
        dict_complete_range_rows[row] = get_complete_range_row(row, 
                                                               row_ranges_between_red_tiles,
                                                               col_ranges_between_red_tiles)

    return dict_complete_range_rows


def compute_area_p1(t1, t2):
    length_vertical = abs(t2[0] - t1[0]) + 1
    length_horizontal = abs(t2[1] - t1[1]) + 1
    return length_vertical*length_horizontal


def compute_area_p2(t1, t2, dict_complete_range_rows):

    # Vérification de la validité du rectangle
    a1 = (min(t1[0], t2[0]), min(t1[1], t2[1])) # NW
    a2 = (min(t1[0], t2[0]), max(t1[1], t2[1])) # NE
    a3 = (max(t1[0], t2[0]), min(t1[1], t2[1])) # SW
    a4 = (max(t1[0], t2[0]), max(t1[1], t2[1])) # SE

    if (
        a1[1] in dict_complete_range_rows[a1[0]]
        and a2[1] in dict_complete_range_rows[a2[0]]
        and a3[1] in dict_complete_range_rows[a3[0]]
        and a4[1] in dict_complete_range_rows[a4[0]]
        ):
        return 0

    # Calcul de l'aire
    if t1[0] == t2[0]:
        # Segment horizontal
        area = max(t1[1], t2[1]) - min(t1[1], t2[1]) + 1
    elif t1[1] == t2[1]:
        # Segment vertical
        area = max(t1[0], t2[0]) - min(t1[0], t2[0]) + 1
    else:
        # Rectangle
        area = (a2[1] - a1[1] + 1) * (a3[0] - a1[0] + 1)

    return area

In [21]:
red_tiles = [(3, 2), (3, 7), (1, 7), (1, 11), (3, 11), (3, 13), (1, 13), (1, 15), (3, 15), (3, 17), (7, 17), (7, 9), (5, 9), (5, 2)]

grid = np.full((9, 18), ".")
for x, y in red_tiles:
    grid[x, y] = "#"

print(grid)
compute_ranges_from_red_tiles(red_tiles, by="row")

[['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '#' '.' '#' '.' '#' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '#' '.' '.' '.' '#' '.' '#' '.' '#' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.' '.']]


{1: range(13, 16), 3: range(15, 18), 5: range(2, 10), 7: range(9, 18)}

In [12]:
def main(input_file, part):

    red_tiles = parse_input(input_file)
    if part == 2:
        dict_complete_range_rows = get_all_complete_range_rows(red_tiles)
    
    max_area = 0
    for t1, t2 in itertools.combinations(red_tiles, 2):
        if part == 1:
            area = compute_area_p1(t1, t2)
        elif part == 2:
            area = compute_area_p2(t1, t2, dict_complete_range_rows)
            # print(t1, t2, area)
        max_area = max(max_area, area)

    return max_area

assert main("example.txt", part=1) == 50
main("input.txt", part=1)

4741848414

In [13]:
assert main("example.txt", part=2) == 24


In [ ]:
red_tiles = parse_input("input.txt")
dict_complete_range_rows = get_all_complete_range_rows(red_tiles)


In [ ]:
from collections import Counter


c = Counter(list([x for x, y in red_tiles]))
c.most_common()


[(96653, 4), (1708, 2), (2016, 2)]

In [ ]:
def compute_ranges_from_red_tiles(red_tiles, by):

    if by == "col":
        red_tiles = [(y, x) for x, y in red_tiles]
    red_tiles = sorted(red_tiles, key=lambda x: x[0])
    counts_row_appear = Counter(list([x for x, y in red_tiles]))

    row_ranges = {}
    for i in range(0, len(red_tiles)-1, 2):
        bottom = min(red_tiles[i][1], red_tiles[i+1][1])
        top = max(red_tiles[i][1], red_tiles[i+1][1])
        row_ranges[red_tiles[i][0]] = range(bottom, top+1)
    return row_ranges

red_tiles = parse_input("example.txt")
compute_ranges_from_red_tiles(red_tiles, by="row")

{1: range(7, 12), 3: range(2, 8), 5: range(2, 10), 7: range(9, 12)}

In [14]:
def compute_ranges_from_red_tiles(red_tiles, by):

    if by == "col":
        red_tiles = [(y, x) for x, y in red_tiles]
    red_tiles = sorted(red_tiles, key=lambda x: x[0])

    row_ranges = {}
    for row in set((x for x, y in red_tiles)):
        row_ranges[row] = []
        for i in range(0, len(red_tiles)-1, 2):
            min_col = min(red_tiles[i][1], red_tiles[i+1][1])
            max_col = max(red_tiles[i][1], red_tiles[i+1][1])
            if red_tiles[i][0] == row:
                row_ranges[row].append(range(min_col, max_col+1))

    return row_ranges

red_tiles, min_row, max_row = parse_input("example.txt")
row_ranges = compute_ranges_from_red_tiles(red_tiles, by="row")
col_ranges = compute_ranges_from_red_tiles(red_tiles, by="col")

row_ranges

{1: [range(7, 12)], 3: [range(2, 8)], 5: [range(2, 10)], 7: [range(9, 12)]}

In [15]:
col_ranges

{9: [range(5, 8)], 2: [range(3, 6)], 11: [range(1, 8)], 7: [range(1, 4)]}

In [ ]:
for row in range(min_row, max_row+1):
    if row not in row_ranges:
        row_ranges[row] = []
        current_row_range = None
    else:
        current_row_range = row_ranges[row][0]
    for col in sorted(col_ranges.keys()):
        print(col, current_row_range
        if col_ranges[col][0].start <= row < col_ranges[col][0].stop and (current_row_range is None or col not in current_row_range):
                row_ranges[row].append(range(col, col+1))
        else:
            if col_ranges[col][0].start <= row < col_ranges[col][0].stop:
                row_ranges[row].append(range(col, col+1))

row_ranges

2 range(7, 12)
7 range(7, 12)
9 range(7, 12)
11 range(7, 12)
2 range(7, 8)
7 range(7, 8)
9 range(7, 8)
11 range(7, 8)
2 range(2, 8)
7 range(2, 8)
9 range(2, 8)
11 range(2, 8)
2 range(2, 3)
7 range(2, 3)
9 range(2, 3)
11 range(2, 3)
2 range(2, 10)
7 range(2, 10)
9 range(2, 10)
11 range(2, 10)
2 range(9, 10)
7 range(9, 10)
9 range(9, 10)
11 range(9, 10)
2 range(9, 12)
7 range(9, 12)
9 range(9, 12)
11 range(9, 12)


{1: [range(7, 12), range(7, 8), range(11, 12), range(7, 8), range(11, 12)],
 3: [range(2, 8),
  range(2, 3),
  range(7, 8),
  range(11, 12),
  range(2, 3),
  range(7, 8),
  range(11, 12)],
 5: [range(2, 10),
  range(2, 3),
  range(9, 10),
  range(11, 12),
  range(2, 3),
  range(9, 10),
  range(11, 12)],
 7: [range(9, 12), range(9, 10), range(11, 12), range(9, 10), range(11, 12)],
 2: [range(7, 8), range(11, 12), range(7, 8), range(11, 12)],
 4: [range(2, 3), range(11, 12), range(2, 3), range(11, 12)],
 6: [range(9, 10), range(11, 12), range(9, 10), range(11, 12)]}

In [76]:
col_ranges

{9: [range(5, 8)], 2: [range(3, 6)], 11: [range(1, 8)], 7: [range(1, 4)]}